In [1]:
import numpy as np
import pandas as pd

import requests

In [2]:
from warnings import filterwarnings
filterwarnings("ignore")

# Download Raw Data from the API

In [3]:
download_url = "http://127.0.0.1:8000/download/raw"
response = requests.get(download_url)
response.status_code

200

In [4]:
data = pd.DataFrame(response.json())
data.head(2)

,Year,Quarter,type,NumberRecords,Devices,Tests,Avg_Avg_U_Kbps,Avg_Avg_D_Kbps,Avg_Lat_Ms,Avg_Pop2005,Rank_Upload,Rank_Download,Rank_Latency
0,2020,1,fixed,"11,913",39923,170080,1368,3492,61,32854159,230,227,60
1,2020,2,fixed,"12,220",39516,194803,991,3130,55,32854159,234,228,73


In [5]:
# Desatinize the None values
for col in data.columns:
    data[col] = data[col].apply(lambda X: None if '' == X else X)

# Feature Selection

**Avg_Pop2005** and **NumberRecords** not necessary for this analysis.

In [6]:
data.drop(columns=["Avg_Pop2005","NumberRecords"],inplace=True)

# Handling Missing Values

In [7]:
data.isnull().sum()

Year              0
Quarter           0
type              0
Devices           0
Tests             0
Avg_Avg_U_Kbps    0
Avg_Avg_D_Kbps    0
Avg_Lat_Ms        0
Rank_Upload       0
Rank_Download     0
Rank_Latency      0
dtype: int64

Data is **cleaned**.

# Handling Duplicates

In [8]:
data.duplicated().unique()

array([False])

Not duplicates in the Data.

# Feature Engineering

In [10]:
data['UploadtoDownloadRatio'] = np.where(data.Avg_Avg_D_Kbps != 0,data.Avg_Avg_U_Kbps / data.Avg_Avg_D_Kbps,0)

In [17]:
data['Year/Quarter'] = data['Year'].astype(str) + "/" + data['Quarter'].astype(str)

# Upload the Processed Data to the API

In [18]:
upload_url = "http://127.0.0.1:8000/upload/processed"
response = requests.post(upload_url,json={"content": data.to_dict(orient="records")})
response.status_code

200

In [19]:
response.text

'{"detail":"Upload Data Successfully"}'